In [1]:
import experiment_utils
from experiment_utils import config

config.experiment_name = "0002_heat_1d"
config.target = "jmlr"
config.debug_mode = True

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import probnum as pn

import linpde_gp

[KeOps] Warning : CUDA libraries not found or could not be loaded; Switching to CPU only.


In [3]:
%matplotlib inline

In [4]:
plt.rcParams.update(config.tueplots_bundle())

In [5]:
rng = np.random.default_rng(24)

## Problem Definition

In [6]:
spatial_domain = linpde_gp.domains.asdomain([-1.0, 1.0])

ibvp = linpde_gp.problems.pde.HeatEquationDirichletProblem(
    t0=0.0,
    T=5.0,
    spatial_domain=spatial_domain,
    alpha=0.1,
    initial_values=linpde_gp.functions.TruncatedSineSeries(
        spatial_domain,
        coefficients=[1.0],
    ),
)

In [7]:
plt_grid = ibvp.domain.uniform_grid((100, 50))

## Prior

In [8]:
lengthscale_t = 2.5
lengthscale_x = 2.0
output_scale = 1.0

In [13]:
u_prior = pn.randprocs.GaussianProcess(
    mean=linpde_gp.functions.Zero(input_shape=(2,)),
    cov=output_scale ** 2 * linpde_gp.randprocs.covfuncs.TensorProduct(
        linpde_gp.randprocs.covfuncs.Matern((), nu=1.5, lengthscales=lengthscale_t),
        linpde_gp.randprocs.covfuncs.Matern((), nu=2.5, lengthscales=lengthscale_x),
    ),
)

AttributeError: `np.float_` was removed in the NumPy 2.0 release. Use `np.float64` instead.

In [14]:
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})

ax.plot_surface(plt_grid[..., 0], plt_grid[..., 1], u_prior.mean(plt_grid))

NameError: name 'u_prior' is not defined

Error in callback <function _draw_all_if_interactive at 0xffff4919e700> (for post_execute), with arguments args (),kwargs {}:


RuntimeError: Failed to process string with tex because latex could not be found

RuntimeError: Failed to process string with tex because latex could not be found

<Figure size 600x370.82 with 1 Axes>

In [ ]:
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})

ax.plot_surface(plt_grid[..., 0], plt_grid[..., 1], ibvp.solution(plt_grid))

### Dirichlet Problem

In [ ]:
N_ic = 5
N_bc = 50

#### Initial conditions

In [ ]:
X_ic = ibvp.initial_domain.uniform_grid(N_ic, inset=1e-6)
Y_ic = ibvp.initial_condition.values(X_ic[..., 1])

u_ic = u_prior.condition_on_observations(Y_ic, X=X_ic)

#### Boundary Conditions

In [ ]:
u_ic_bc = u_ic

for bc in ibvp.boundary_conditions:
    X_bc = bc.boundary.uniform_grid(N_bc)
    Y_bc = bc.values(X_bc)

    u_ic_bc = u_ic_bc.condition_on_observations(Y_bc, X=X_bc)

#### Prior with Initial and Boundary Conditions

In [ ]:
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})

ax.plot_surface(plt_grid[..., 0], plt_grid[..., 1], u_ic_bc.mean(plt_grid))

### Conditioning on the PDE

In [ ]:
N_pde = (100, 20)

In [ ]:
# Before
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})

ax.plot_surface(plt_grid[..., 0], plt_grid[..., 1], ibvp.pde.diffop(u_ic_bc).mean(plt_grid))

In [ ]:
X_pde = ibvp.domain.uniform_grid(N_pde)
Y_pde = ibvp.pde.rhs(X_pde)

In [ ]:
u_ic_bc_pde = u_ic_bc.condition_on_observations(
    Y_pde,
    X=X_pde,
    L=ibvp.pde.diffop,
)

In [ ]:
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})

ax.plot_surface(plt_grid[..., 0], plt_grid[..., 1], u_ic_bc_pde.mean(plt_grid))
ax.set_xlabel("Time (s)")
ax.set_ylabel("Location (cm)")
ax.set_zlabel("Temperature (°C)")

experiment_utils.savefig("heat_posterior")

In [ ]:
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})

ax.plot_surface(plt_grid[..., 0], plt_grid[..., 1], ibvp.pde.diffop(u_ic_bc_pde).mean(plt_grid))

In [ ]:
np.average(np.abs(u_ic_bc_pde.mean(plt_grid) - ibvp.solution(plt_grid)))

### Generate Animation

In [ ]:
from matplotlib import animation

In [ ]:
fig, ax = plt.subplots(dpi=500)

def frame_fn(frame_idx):
    txs = plt_grid[frame_idx, :, :]

    ax.cla()

    mean = u_ic_bc_pde.mean(txs)
    std = u_ic_bc_pde.std(txs)

    ax.plot(txs[:, 1], mean, label=r"$\mathrm{u} \mid \mathrm{PDE}, \mathrm{BC}$")
    ax.fill_between(
        txs[:, 1],
        mean - 1.96 * std,
        mean + 1.96 * std,
        alpha=.3,
    )

    ax.plot(txs[:, 1], ibvp.solution(txs), label=r"$u^\star$")

    ax.set_ylim(-0.01, 1.2)
    ax.set_xlabel("Location (cm)")
    ax.set_ylabel("Temperature (°C) ")
    ax.set_title(f"t = {plt_grid[frame_idx, 0, 0]:.2f} s")
    ax.legend(loc="upper right")

anim = animation.FuncAnimation(
    fig,
    frame_fn,
    frames=plt_grid.shape[0],
    interval=10,
    repeat_delay=4000,
    blit=False,
)

In [ ]:
experiment_utils.savefig("heat_anim_firstframe", fig=fig)

In [ ]:
from IPython.display import HTML

HTML(anim.to_jshtml())

In [ ]:
anim_path = experiment_utils.config.experiment_results_path / "heat_anim"

if anim_path.is_dir():
    import shutil

    shutil.rmtree(anim_path)

anim_path.mkdir(exist_ok=True)

anim.save(anim_path / "{}.pdf", linpde_gp.utils.plotting.PDFWriter())

In [ ]:
try:
    anim.save(
        experiment_utils.config.experiment_results_path / "heat_anim.mp4",
        animation.FFMpegWriter(fps=20),
    )
except FileNotFoundError:
    import warnings
    warnings.warn("FFmpeg not installed")